In [2065]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap

import warnings
warnings.filterwarnings('ignore')

from scipy.special import binom

In [2066]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        #deme = name.split(".")[1]# the syntax here is "Ne_region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[1]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)


In [2067]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            median_linear = math.exp(median_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, "median_Ne_linear": median_linear, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [2068]:
#calculating transmission rate
def generate_summary_diff_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
            next_interval = int(interval)+4 #averaging over two weeks to reduce noise
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
           
            try:
                new_df["Ne"+ ".diff." + str(interval)] = (52/4)*(input_df[i].astype("float") - input_df["Ne"+"." + str(next_interval)].astype('float'))
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [2158]:
def generate_local_and_intro_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/14
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            new_df["rt" +  "." + str(interval)] = (1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))
            

            
    return(new_df)

In [2070]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_rt_summary_df(input_df):
    
    
    new_df = pd.DataFrame()
    count = 0
    for i in input_df.columns.tolist():
        #print(i)
        if "rt" in i:
            if count %1 == 0:
                interval = i.split(".")[1]
                #interval = i.split(".")[2]
                local_series = input_df[i].astype('float').to_numpy()
                mean_percent = local_series.mean()
                median_percent = np.median(local_series)

                hpd_95 = az.hdi(local_series, 0.95)
                lower_hpd_log_95 = hpd_95[0]
                upper_hpd_log_95 = hpd_95[1]
                hpd_50 = az.hdi(local_series, 0.50)
                lower_hpd_log_50 = hpd_50[0]
                upper_hpd_log_50 = hpd_50[1]

                try:
                    local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, "median_percent":median_percent,
                                                       "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                       "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                    new_df = new_df.append(local_df)
                except:
                    pass
            count+=1

    return(new_df)

In [2130]:

log_file_path = "../simulations/beast_results/simmulticoal_4_all.log"

In [2072]:
ne_diff_summary = generate_summary_diff_df(Ne_df)


In [2166]:
uninfectious_rate = 365/4.5

#taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
incubation_period = 365/8

In [2167]:
seir_growth_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)
seir_growth_rate.head()

,Ne.diff.1,Ne.diff.2,Ne.diff.3,Ne.diff.4,Ne.diff.5,Ne.diff.6,Ne.diff.7,Ne.diff.8,Ne.diff.9,Ne.diff.10,...,Ne.diff.25,Ne.diff.26,Ne.diff.27,Ne.diff.28,Ne.diff.29,Ne.diff.30,Ne.diff.31,Ne.diff.32,Ne.diff.33,Ne.diff.34
0,2075.331253,-13.290490,112.083260,2075.343548,-3.809273,427.511735,594.789213,205.768320,10.516746,65.141100,...,108.884382,0.622137,122.419123,298.720644,74.800279,577.425000,1901.026883,619.816855,47.433550,10.223636
1,1499.348740,-12.774735,431.050394,1462.848366,-28.089796,77.492317,112.323309,543.425397,-14.864906,141.730582,...,27.872205,261.832213,16.791799,501.095161,122.741124,20.579904,1144.777304,438.005890,69.046067,267.821676
2,1092.635032,-25.241483,155.736162,1825.275681,-17.182837,349.496183,117.336752,292.438813,34.614242,-6.674546,...,-11.986541,204.604381,131.656526,1338.358495,146.538592,230.115897,638.875809,309.023205,91.921275,499.547781
3,1838.981795,36.261765,171.563442,66.445489,-28.959989,133.682420,53.210834,371.860380,9.328985,206.073406,...,25.068305,45.860182,9.385490,922.571611,153.051124,491.221235,1245.617109,369.131506,139.428569,440.419373
4,707.306936,-19.792788,96.960858,-23.772547,-16.794379,672.546068,385.031270,344.405177,109.103581,12.643103,...,186.576780,-6.342613,-11.555712,195.151936,155.852588,367.422934,703.498507,316.348068,37.786639,849.206822


In [2075]:
#read in log file
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

#remove burn-in
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))

burnin_percent = 0.3
print(len(Ne_df))
rows_to_remove = int(len(Ne_df)* burnin_percent)
Ne_df = Ne_df.iloc[rows_to_remove:]

print(len(Ne_df))
Ne_df = Ne_df.reset_index()

ne_summary = generate_summary_df(Ne_df)

#format time intervals
test = ne_summary
test['days'] = (test.interval.astype(int))*7
test['date'] = dt.strptime("2023-01-14",  "%Y-%m-%d") - test.days.map(timedelta)

test["new_interval"] = (((test.interval.iloc[::-1]).astype(int))*7 )-7

#add smoothing 
columns = ["mean_Ne_log","mean_Ne_linear",  "median_Ne_linear","median_Ne_log", "upper_hpd_log_50","lower_hpd_log_50", "upper_hpd_linear_50","lower_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"]
for column in columns:
    # Create a new column for the moving average
    test[f'{column}_MA'] = test[column].rolling(5, min_periods =1).mean()


5001
5001
3501


In [2100]:
#plot
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
log_line_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.8, clip = True).encode(
    alt.X('new_interval', ),
    alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="Effective Population Size", grid=False), scale = alt.Scale(domain=[-3,8])),
    alt.Y2('lower_hpd_log_50_MA' ),
    
).properties(
    width=850,
    height=300
)
log_band_ne = alt.Chart(test).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
    alt.X('new_interval', ),
    alt.Y('median_Ne_log_MA',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),

    
).properties(
    width=850,
    height=300
)

log_line_ne  + log_band_ne

alt.LayerChart(...)

In [2101]:
simulation_df = pd.read_json("../simulations/eir_4.json", orient = "index")
simulation_df = simulation_df.T
simulation_df.I[simulation_df.I < 0.1] = 0.1
simulation_df.I = simulation_df.I.astype("float")
simulation_df["log_I"] = simulation_df.I.apply(np.log)
simulation_df_copy = simulation_df.copy()
simulation_df_copy["diff"] = simulation_df_copy.log_I.diff() *52
simulation_df

,R,samp2,samp,t,E,IntroCount,sim,I,Iinit,log_I
0,0.0,0.0,0.0,0.0,0.0,0.0,populationEndConditions,0.1,1.0,-2.302585
1,0.0,0.0,0.0,0.001522,0.0,1.0,postSimConditions,1.0,1.0,0.000000
2,0.0,0.0,0.0,0.002102,0.0,2.0,stepper,2.0,1.0,0.693147
3,0.0,0.0,0.0,0.002197,0.0,3.0,lineageEndConditions,3.0,1.0,1.098612
4,0.0,0.0,0.0,0.002678,0.0,4.0,leafCountEndConditions,4.0,1.0,1.386294
...,...,...,...,...,...,...,...,...,...,...
4059,1214.0,0.0,639.0,0.704329,20.0,1412.0,NaN,25.0,1.0,3.218876
4060,1214.0,0.0,639.0,0.704463,20.0,1413.0,NaN,26.0,1.0,3.258097
4061,1214.0,0.0,640.0,0.704776,20.0,1413.0,NaN,25.0,1.0,3.218876
4062,1214.0,0.0,640.0,0.704826,20.0,1414.0,NaN,26.0,1.0,3.258097


In [2102]:
simulation_df["new_time"] = simulation_df.t *366
line_simulation = alt.Chart(simulation_df).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "orange").encode(
    alt.X('new_time', axis=alt.Axis(title="")),
    alt.Y('log_I',axis=alt.Axis(title="Simulated prevalence", grid=False)),
    
    
).properties(
    width=850,
    height=300)
line_simulation

alt.Chart(...)

In [2103]:
(line_simulation + log_line_ne + log_band_ne).resolve_scale(x = "shared", y = "shared")

alt.LayerChart(...)

In [2119]:
## calculate Rt

In [2159]:
rt_local_and_intro_df = generate_local_and_intro_rt(ne_diff_summary)
summary_rt_local_and_intro_df = generate_rt_summary_df(rt_local_and_intro_df)
summary_rt_local_and_intro_df["new_interval"] = (((summary_rt_local_and_intro_df.interval.iloc[::-1]).astype(int))*7 )-7

In [2160]:
columns = ["mean_percent","median_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_rt_local_and_intro_df[f'{column}_MA'] = summary_rt_local_and_intro_df[column].rolling(4, min_periods =1).mean()

In [2161]:
summary_rt_local_and_intro_df.head()

,interval,mean_percent,median_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,mean_percent_MA,median_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,1,28.693404,22.470384,76.604074,-0.093396,24.464400,3.296550,231,28.693404,22.470384,76.604074,-0.093396,24.464400,3.296550
0,2,4.353771,1.541644,17.218072,-0.358135,1.541644,-0.358135,224,16.523588,12.006014,46.911073,-0.225765,13.003022,1.469208
0,3,1.278361,1.107053,3.067918,-0.239057,1.430897,0.393405,217,11.441846,8.373027,32.296688,-0.230196,9.145647,1.110607
0,4,6.508848,4.836102,19.158606,-0.358135,4.836102,-0.358135,210,10.208596,7.488796,29.012167,-0.262180,8.068261,0.743421
0,5,2.259704,0.082764,12.497147,-0.358135,0.082764,-0.358135,203,3.600171,1.891891,12.985436,-0.328365,1.972852,-0.170250


In [2162]:
simulation_df_copy["whole_time"] = np.ceil(simulation_df_copy.t *366)

#define to week number
counter = 0
simulation_df_copy["week_time"] = np.nan
for index, row in daily_sim_cases.iterrows():
    if row.whole_time %7 == 0:
        simulation_df_copy.loc[index,"week_time"] = counter 
        counter +=1
        
#backfill
simulation_df_copy = simulation_df_copy.bfill()

# aggregate based on weeks
weekly_sim_cases = simulation_df_copy.groupby(["week_time"])["diff"].sum().reset_index()

weekly_sim_cases.head()

,week_time,diff
0,0.0,119.734425
1,1.0,176.862264
2,2.0,36.043653
3,3.0,21.084186
4,4.0,-6.124718


In [2163]:
#calculate Rt based on cases
incubation_period = 365/8
uninfectious_rate = 365/4.5

weekly_sim_cases["Rt"] = (1+ (weekly_sim_cases["diff"] / 12.5))
weekly_sim_cases.week_time = weekly_sim_cases.week_time *7

In [2164]:

line_rt_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "blue").encode(
    alt.X('week_time', axis=alt.Axis(title="")),
    alt.Y('Rt',axis=alt.Axis(title="Rt", grid=False), scale = alt.Scale(domain = [0.3, 10])),
    
    
).properties(
    width=850,
    height=300)
line_rt_sim

alt.Chart(...)

In [2185]:
summary_rt_local_and_intro_df.head()

,interval,mean_percent,median_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,mean_percent_MA,median_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,1,28.693404,22.470384,76.604074,-0.093396,24.464400,3.296550,231,28.693404,22.470384,76.604074,-0.093396,24.464400,3.296550
0,2,4.353771,1.541644,17.218072,-0.358135,1.541644,-0.358135,224,16.523588,12.006014,46.911073,-0.225765,13.003022,1.469208
0,3,1.278361,1.107053,3.067918,-0.239057,1.430897,0.393405,217,11.441846,8.373027,32.296688,-0.230196,9.145647,1.110607
0,4,6.508848,4.836102,19.158606,-0.358135,4.836102,-0.358135,210,10.208596,7.488796,29.012167,-0.262180,8.068261,0.743421
0,5,2.259704,0.082764,12.497147,-0.358135,0.082764,-0.358135,203,3.600171,1.891891,12.985436,-0.328365,1.972852,-0.170250


In [2165]:
band_rt = alt.Chart(summary_rt_local_and_intro_df).mark_area(interpolate='monotone', opacity = 0.5, clip = True, color = "orange").encode(
    alt.X('new_interval', axis=alt.Axis(title="")),
    alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="Rt", grid=False)),
    alt.Y2("lower_hpd_log_50_MA")
    
    
).properties(
    width=850,
    height=300)

line_rt = alt.Chart(summary_rt_local_and_intro_df).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
    alt.X('new_interval', axis=alt.Axis(title="")),
    alt.Y('median_percent_MA',axis=alt.Axis(title="Rt", grid=False)),

    
    
).properties(
    width=850,
    height=300)

(band_rt + line_rt+ line_rt_sim).resolve_scale(y ="shared")

alt.LayerChart(...)

In [2118]:
## calculate percentage of cases due to introduction

In [2168]:
def read_in_forward_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "immigrationRate" in col:
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                        name = line.split("\t")[n]
                        interval = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        #interval = name.split(".")[2]
                       
                        mig_key[n] = name
                        mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                    
                
                
                
    return(mig_rates_dict)

In [2169]:
def generate_percent_intro_df(input_df):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "immigrationRate" in i:
        
            interval = i.split(".")[1]
            #deme = i.split(".")[2]

            try:
                temp_df["total."+ str(interval)] = seir_growth_rate["Ne.diff." + str(interval)].astype("float") +  input_df[i].astype("float").apply(np.exp)
                new_df["intro.percent"+ "." + str(interval)] = input_df[i].astype("float").apply(np.exp).div(temp_df["total."+ str(interval)], axis = 0) 

            except KeyError: #this was added because not all regions have equal time periods for their epidemics and it was throwing an error everytime it had to switch deme
                pass 
                  
    return(new_df)

In [2170]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            interval = i.split(".")[2]
            #interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({ "interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)


In [2171]:
migration_rates_f = read_in_forward_migration_rates_mascot(log_file_path)
mig_df_f = pd.DataFrame.from_dict(migration_rates_f)

#remove burnin
burnin_percent = 0.3
print(len(mig_df_f))
rows_to_remove = int(len(mig_df_f)* burnin_percent)
mig_df_f = mig_df_f.iloc[rows_to_remove:]

print(len(mig_df_f))
mig_df_f = mig_df_f.reset_index()
mig_df_f.head()

1264
885


,index,sample,immigrationRate.1,immigrationRate.2,immigrationRate.3,immigrationRate.4,immigrationRate.5,immigrationRate.6,immigrationRate.7,immigrationRate.8,...,immigrationRate.28,immigrationRate.29,immigrationRate.30,immigrationRate.31,immigrationRate.32,immigrationRate.33,immigrationRate.34,immigrationRate.35,immigrationRate.36,immigrationRate.37
0,379,18950000,4.475225245129132,3.971435152758826,4.070106116141131,3.5435799825311807,3.765084816603576,3.5479679746437265,3.629970387366619,3.4261584985729425,...,2.7124404581864505,3.656849813330297,3.535931889830322,3.393913031269265,3.9530850860240117,3.601709902998919,3.8871837002886895,4.535603155920583,5.539125575190765,5.392933212023119
1,380,19000000,4.213268630540959,4.217405835527005,3.9713032330431703,3.929878152672807,4.1913548388482225,3.6407928355213732,3.8411527274848694,3.8694331631488157,...,3.0165529937641518,3.494024154827591,3.659075302679914,3.2121281698578827,3.7380720702863655,3.623504898577168,3.783598240037627,4.340074674656389,5.099173702572974,5.026330776015945
2,381,19050000,4.195383401555008,3.688591429447584,4.184532429992743,3.7879918299242528,4.236820650780359,3.5015834971419753,4.216856736600312,3.7698713075860493,...,2.51231277032843,3.3052209896887317,3.4353839619180144,2.964315169547319,4.026436732029131,3.5751597362875085,3.3418179214857204,4.48322902886098,5.104556381821502,6.398209822224047
3,382,19100000,4.349636058932156,4.132984069928846,3.635777519250339,4.200337494223944,3.571422031354898,3.77209293923503,3.5408329200280315,3.4726025045444615,...,3.1198012162091096,3.103327552468917,3.7064076683617633,3.4902664346537478,4.21186014165674,4.068738525762182,3.9611064514807186,4.336478484201948,5.011869495443437,4.774932199701903
4,383,19150000,3.9126360653594614,3.8578840217426813,4.050517233583089,3.727057700083146,3.732162513281409,3.9429711863426804,3.982675584275632,3.640221374176707,...,3.199248109268815,3.6800265451374137,4.020671640347107,3.9659626748358567,3.2274536003159437,3.13880093732044,3.8126262621264226,4.057466794926745,4.977567679898902,5.822174117704034


In [2172]:
percent_df = generate_percent_intro_df(mig_df_f)
percent_df.head()



,intro.percent.1,intro.percent.2,intro.percent.3,intro.percent.4,intro.percent.5,intro.percent.6,intro.percent.7,intro.percent.8,intro.percent.9,intro.percent.10,...,intro.percent.25,intro.percent.26,intro.percent.27,intro.percent.28,intro.percent.29,intro.percent.30,intro.percent.31,intro.percent.32,intro.percent.33,intro.percent.34
0,0.040596,1.334183,0.343184,0.016394,1.096785,0.075159,0.059623,0.130041,0.717505,0.503410,...,0.236648,0.986544,0.154391,0.048014,0.341195,0.056113,0.015425,0.077534,0.435949,0.826709
1,0.043127,1.231920,0.109591,0.033626,1.738768,0.329735,0.293130,0.081028,1.254188,0.269735,...,0.493911,0.068468,0.602158,0.039157,0.211476,0.653568,0.021231,0.087531,0.351770,0.141035
2,0.057272,2.711635,0.296581,0.023626,1.330409,0.086676,0.366284,0.129162,0.395848,1.219066,...,4.306266,0.077458,0.064823,0.009131,0.156822,0.118867,0.029444,0.153556,0.279736,0.053561
3,0.040414,0.632329,0.181061,0.500989,5.383131,0.245386,0.393308,0.079738,0.824352,0.178029,...,0.444253,0.314416,0.773886,0.023954,0.127034,0.076528,0.025653,0.154558,0.295501,0.106536
4,0.066061,1.717852,0.371966,2.336730,1.672449,0.071220,0.122319,0.099607,0.277814,0.782372,...,0.132571,1.533277,1.939206,0.111597,0.202800,0.131719,0.069778,0.073823,0.379151,0.050610


In [2179]:
summary_percent_df = generate_summary_df(percent_df)
summary_percent_df = summary_percent_df.reset_index()



In [2221]:
#making sure that any numbers >1 are excluded as outliers from stochastic noise
percent_df =pd.DataFrame(np.where(percent_df <1, percent_df, 1), columns=percent_df.columns )
percent_df =pd.DataFrame(np.where(percent_df >0, percent_df, 0), columns=percent_df.columns )

#format dates
summary_percent_df['days'] = (summary_percent_df.interval.astype(int)-3)*7 #the 1.5 adjustment is made due to the fact that we take the difference of Nes over three weeks 
summary_percent_df['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - summary_percent_df.days.map(timedelta)

summary_percent_df.head()





,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,days,date,mean_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,0,1,0.762604,1.0,0.015898,1.0,1.0,-14,2024-09-26,0.762604,1.0,0.015898,1.0,1.0
1,0,2,0.865661,1.0,0.081369,1.0,1.0,-7,2024-09-19,0.814132,1.0,0.048634,1.0,1.0
2,0,3,0.846895,1.0,0.225741,1.0,1.0,0,2024-09-12,0.825053,1.0,0.107670,1.0,1.0
3,0,4,0.807979,1.0,0.043507,1.0,1.0,7,2024-09-05,0.820785,1.0,0.091629,1.0,1.0
4,0,5,0.906296,1.0,0.132197,1.0,1.0,14,2024-08-29,0.856708,1.0,0.120704,1.0,1.0


In [2222]:
columns = ["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_percent_df[f'{column}_MA'] = summary_percent_df[column].rolling(4, min_periods =1).mean()


In [2225]:
summary_percent_df.head()

,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,days,date,mean_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,0,1,0.762604,1.0,0.015898,1.0,1.0,-14,2024-09-26,0.762604,1.0,0.015898,1.0,1.0
1,0,2,0.865661,1.0,0.081369,1.0,1.0,-7,2024-09-19,0.814132,1.0,0.048634,1.0,1.0
2,0,3,0.846895,1.0,0.225741,1.0,1.0,0,2024-09-12,0.825053,1.0,0.107670,1.0,1.0
3,0,4,0.807979,1.0,0.043507,1.0,1.0,7,2024-09-05,0.820785,1.0,0.091629,1.0,1.0
4,0,5,0.906296,1.0,0.132197,1.0,1.0,14,2024-08-29,0.856708,1.0,0.120704,1.0,1.0


In [2223]:
line1 = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_50_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
    alt.Y2('upper_hpd_log_50_MA' )
).properties(
    width=850,
    height=300
)

band = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_95_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
    alt.Y2('upper_hpd_log_95_MA' )
).properties(
    width=850,
    height=300
)

line1
#intro_plot = (line1 + band).resolve_scale(y= "shared") ; intro_plot

alt.Chart(...)

In [ ]:
## now calculate the true percentage of cases due to intros from simulations

In [2186]:
simulation_df["whole_time"] = np.ceil(simulation_df.t *366)

In [2201]:
daily_sim_cases = simulation_df.groupby(["whole_time"]).agg({'IntroCount':'mean','I':'mean'}).reset_index()


In [2208]:
daily_sim_cases["diff_intro"] = daily_sim_cases.IntroCount.diff()
daily_sim_cases["diff_I"] = daily_sim_cases.I.diff()

daily_sim_cases["diff_I"][daily_sim_cases["diff_I"] < 0 ] = 0
daily_sim_cases["diff_intro"][daily_sim_cases["diff_intro"] < 0 ] = 0

In [2212]:
counter = 0
daily_sim_cases["week_time"] = np.nan
for index, row in daily_sim_cases.iterrows():
    if row.whole_time %7 == 0:
        daily_sim_cases.loc[index,"week_time"] = counter 
        counter +=1

In [2213]:
daily_sim_cases = daily_sim_cases.bfill()

In [2215]:
weekly_sim_cases = daily_sim_cases.groupby(["week_time"]).agg({'diff_intro':'mean','diff_I':'mean'}).reset_index()

In [2217]:
weekly_sim_cases["percent_intro"] = weekly_sim_cases.diff_intro/(weekly_sim_cases.diff_intro+ weekly_sim_cases.diff_I)

In [2218]:
percent_intro_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "black").encode(
    alt.X('week_time', axis=alt.Axis(title="")),
    alt.Y('percent_intro',axis=alt.Axis(title="percent of cases due to introductions", grid=False)),
    
    
).properties(
    width=850,
    height=300)

percent_intro_sim


alt.Chart(...)

In [2219]:
line1 = alt.Chart(summary_percent_df).mark_line(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('mean_percent_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
   
).properties(
    width=850,
    height=300
)
line1 +percent_intro_sim

alt.LayerChart(...)